## Solving bidomain equations on the ventricle with modified FitzHugh-Nagumo model

### Importing required libraries

In [1]:
%cd ..
from src.dynamics_models import *
from src.cell_models import *
from src.utils import *

/home/drvinko/Desktop/Modeliranje-biofizike-srcanog-tkiva


### Defining a domain, cell model and dynamics model and visualizing initial data

In [2]:
domain = heart_ventricle()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Circle)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 50%] Meshing curve 6 (Ellipse)
Info    : [ 60%] Meshing curve 7 (Ellipse)
Info    : [ 80%] Meshing curve 9 (Ellipse)
Info    : [ 90%] Meshing curve 10 (Ellipse)
Info    : [100%] Meshing curve 11 (Line)
Info    : Done meshing 1D (Wall 0.00852049s, CPU 0.008589s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (Surface of Revolution, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 3 (Surface of Revolution, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.179591s, CPU 0.179617s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 3157 nodes...

In [3]:
class Model(BidomainModel):
    def initial_V_m(self):
        def value(x):
            mu = -7
            alpha = 0.1
            return -85 + 35 / (1 + np.exp((x[2] - mu) / alpha))

        self.V_m_n.interpolate(value)
        return self.V_m_n

    def conductivity(self):
        # Muscle fibres
        x_c, y_c = 0.2, 1
        self.fibers = ufl.as_vector(
            [
                -(self.x[1] - y_c)
                / ufl.sqrt((self.x[0] - x_c) ** 2 + (self.x[1] - y_c) ** 2),
                (self.x[0] - x_c)
                / ufl.sqrt((self.x[0] - x_c) ** 2 + (self.x[1] - y_c) ** 2),
                0,
            ]
        )
        self.normals = ufl.as_vector([0, 0, 1])

        # Healthy conductivities
        self.M_i = (
            self.SIGMA_IT * ufl.Identity(3)
            + (self.SIGMA_IL - self.SIGMA_IT) * ufl.outer(self.fibers, self.fibers)
            + (self.SIGMA_IN - self.SIGMA_IT) * ufl.outer(self.normals, self.normals)
        )
        self.M_e = (
            self.SIGMA_ET * ufl.Identity(3)
            + (self.SIGMA_EL - self.SIGMA_ET) * ufl.outer(self.fibers, self.fibers)
            + (self.SIGMA_EN - self.SIGMA_ET) * ufl.outer(self.normals, self.normals)
        )

In [4]:
cell_model = ReparametrizedFitzHughNagumo(domain)
model = Model(domain, cell_model)

In [5]:
plot_function(
    model.V_m_n,
    save_to="ventricle_initial_V_m.pdf",
    cmap = 'PiYG'
)

Widget(value='<iframe src="http://localhost:37221/index.html?ui=P_0x7f3425df8610_0&reconnect=auto" class="pyvi…

### Solving equations with given parameters

In [6]:
model.solve(
    T=2000,
    steps=2000,
    save_to="ventricle.mp4",
    checkpoints=[500, 1000, 1500, 2000],
    checkpoint_file="ventricle",
)

Solving problem: 100%|██████████| 2000/2000 [1:10:04<00:00,  2.10s/it]


### Plotting fiber orientations

In [7]:
x_c, y_c = 0, 0
plot_vector_field(
    domain,
    lambda x: (
        -(x[1] - y_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        (x[0] - x_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        0,
    ),
    0.02,
    0.4,
    save_to="ventricle_fibers.pdf",
)

Widget(value='<iframe src="http://localhost:37221/index.html?ui=P_0x7f33bc0eced0_1&reconnect=auto" class="pyvi…

ERROR:wslink.protocol:Exception raised
ERROR:wslink.protocol:KeyError('374bc1b9e2137b242f1206720652eed7_12622d')
ERROR:wslink.protocol:Traceback (most recent call last):
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/wslink/protocol.py", line 332, in onMessage
    results = func(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/protocols/local_rendering.py", line 33, in get_array
    self.context.get_cached_data_array(data_hash, binary)
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/serializers/synchronization_context.py", line 30, in get_cached_data_array
    cache_obj = self.data_array_cache[p_md5]
                ~~~~~~~~~~~~~~~~~~~~~^^^^^^^
KeyError: '374bc1b9e2137b242f1206720652eed7_12622d'

